In [1]:
from fixedpoint import FixedPoint
from pynq import Overlay
from pynq import DefaultIP

In [2]:
class linRegPredDriver(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)
        
    bindto = ['xilinx.com:hls:calcXTX:1.0']
    
    #given array[8] of floats, x and y, will perform linear regression and return best fit y's for same input x's
    def linReg(self, xFloat, yFloat):
        
        xFixed = [FixedPoint(x, True, 16, 16) for x in xFloat]
        yFixed = [FixedPoint(y, True, 16, 16) for y in yFloat]
        
        intBits = []
        for x in xFixed:
            intBits.append(int(x.bits))

        for y in yFixed:
            intBits.append(int(y.bits))

        for i in range(0, 16, 1):
            self.write(i*8+16, intBits[i])
        self.write(0x00, 1)
        
        qformat = {'signed': True, 'm': 16, 'n': 16}
        retFixed = FixedPoint(0, **qformat)
        yOut = []
        for i in range(0, 8, 1):
            out = self.read((i*16)+0x90)
            retFixed.from_string(hex(out))
            yOut.append(float(retFixed))
            
        return yOut

In [3]:
overlay = Overlay('/home/xilinx/pynq/overlays/linRegOverlays/directImp_v3.bit')
#overlay?

In [4]:
X_float = [-1.1, 0.1, 1.2, 2.3, 3.1, 4.1, 4.8, 5.7]
y_float = [-1.7, 2.4, 5.0, 7.3, 10.9, 12.5, 16.2, 19.7]

In [5]:
%%time
results = overlay.linReg.linReg(X_float, y_float)

CPU times: user 11.6 ms, sys: 0 ns, total: 11.6 ms
Wall time: 11.2 ms


In [6]:
print(results)

[-1.9049530029296875, 1.70611572265625, 5.0161895751953125, 8.326309204101562, 10.733673095703125, 13.74285888671875, 15.849273681640625, 18.557525634765625]
